In [1]:
# Importing libraries
import pandas as pd
import emoji
import re
import string
import json
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

# tensorflow kubernetes
p_list = ['facebook', 'tensorflow', 'microsoft', 'bitcoin','opencv',
          'ansible','flutter','kubernetes','roslyn','typescript', 'dartlang','integrate']
project_name = 'flutter'
Clean_Method = "CM1"


# Loading data from CSV files
train_data = pd.read_csv('../'+Clean_Method+'/data/csvdata/'+project_name+'_train.csv')
test_data = pd.read_csv('../'+Clean_Method+'/data/csvdata/'+project_name+'_test.csv')

print('학습데이터 : ../'+Clean_Method+'/data/csvdata/'+project_name+'_train.csv')
print('테스트 데이터 : ../'+Clean_Method+'/data/csvdata/'+project_name+'_test.csv')

학습데이터 : ../CM1/data/csvdata/flutter_train.csv
테스트 데이터 : ../CM1/data/csvdata/flutter_test.csv


In [2]:
print(train_data.shape)
print(test_data.shape)

(300, 6)
(300, 6)


In [3]:
import os
os.environ['HF_TOKEN']="API_KEY
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")
# model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")
                  

In [4]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [5]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer, SFTConfig
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split

In [6]:

print(f"pytorch version {torch.__version__}")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"working on {device}")


pytorch version 2.2.2
working on cuda:0


In [7]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [8]:

def generate_prompt(data_point):
    return f"<|begin_of_text|><|start_header_id|>system<|end_header_id|> GitHub Issue Report Classifier<|eot_id|><|start_header_id|>user<|end_header_id|>"+str(data_point["content"]) +"<|eot_id|><|start_header_id|>assistant<|end_header_id|>"+str(data_point["label"])+"<|eot_id|>".strip()
    # return f"<|begin_of_text|><|start_header_id|>system<|end_header_id|> GitHub Issue Report Classifier <|eot_id|><|start_header_id|>user<|end_header_id|> Classify, IN ONLY 1 WORD, the following GitHub issue as 'feature', 'bug', or 'question' based on its title and body:"+str(data_point["content"])+"<|eot_id|><|start_header_id|>assistant<|end_header_id|>"+str(data_point["label"])+"<|eot_id|><|end_of_text|>".strip()



   
def generate_test_prompt(data_point):
     return f"<|begin_of_text|><|start_header_id|>system<|end_header_id|> GitHub Issue Report Classifier<|eot_id|><|start_header_id|>user<|end_header_id|>"+str(data_point["content"])+"<|eot_id|><|start_header_id|>assistant<|end_header_id|>".strip()
      # return f"<|begin_of_text|><|start_header_id|>system<|end_header_id|> GitHub Issue Report Classifier <|eot_id|><|start_header_id|>user<|end_header_id|> Classify, IN ONLY 1 WORD, the following GitHub issue as 'feature', 'bug', or 'question' based on its title and body:"+str(data_point["content"])+"<|eot_id|><|start_header_id|>assistant<|end_header_id|>".strip()


X_train = pd.DataFrame(train_data.apply(generate_prompt, axis=1), 
                       columns=["text"])



y_true = test_data.label
X_test = pd.DataFrame(test_data.apply(generate_test_prompt, axis=1), columns=["text"])
train_data = Dataset.from_pandas(X_train)

print(X_train.shape)
print(y_true.shape)
print(X_test.shape)
print(train_data)

print(X_train.head(2))
print(X_test.head(2))


(300, 1)
(300,)
(300, 1)
Dataset({
    features: ['text'],
    num_rows: 300
})
                                                text
0  <|begin_of_text|><|start_header_id|>system<|en...
1  <|begin_of_text|><|start_header_id|>system<|en...
                                                text
0  <|begin_of_text|><|start_header_id|>system<|en...
1  <|begin_of_text|><|start_header_id|>system<|en...


In [9]:
def evaluate(y_true, y_pred):
    labels =  ["bug", "feature", "question"]
    mapping = {'question': 2, 'feature': 1, 'bug': 0}
    def map_func(x):
        return mapping.get(x, 1)
    
    y_true = np.vectorize(map_func)(y_true)
    y_pred = np.vectorize(map_func)(y_pred)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')
    
    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels
    
    for label in unique_labels:
        label_indices = [i for i in range(len(y_true)) 
                         if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')
        
        
    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)
    
    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1, 2])
    print('\nConfusion Matrix:')
    print(conf_matrix)
    
    df = pd.DataFrame(conf_matrix)  # 열 이름 (예측 값)
# CSV 파일로 저장
#     df.to_csv("../CM1/result/"+project_name+"_confusion_matrix.csv", index=True)


In [10]:
model_name = 'meta-llama/Meta-Llama-3.1-8B-Instruct'
# 여기
# model_name="../CM1/model/"+project_name+"-llama-classifier/checkpoint-525"   
# token_name="../CM1/model/"+project_name+"-llama-classifier/checkpoint-525"  
max_seq_length = 1024 #2048
                                            
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device,
    torch_dtype=compute_dtype,
    quantization_config=bnb_config, 
)
tokenizer = AutoTokenizer.from_pretrained(model_name, max_seq_length=max_seq_length)
tokenizer.pad_token_id = tokenizer.eos_token_id


model.config.use_cache = False
model.config.pretraining_tp = 4

# print(tokenizer.eos_token_id)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [11]:
def predict(X_test, model, tokenizer):
    y_pred = []
    for i in tqdm(range(len(X_test))):
        try:
            prompt = X_test.iloc[i]["text"]
            pipe = pipeline(task="text-generation", 
                            model=model, 
                            tokenizer=tokenizer, 
                            max_new_tokens = 1, 
                            temperature = 0.1,
                        )
            result = pipe(prompt)
            answer = result[0]['generated_text'].split("<|end_header_id|>")[-1]
            
            if "question" in answer:
                y_pred.append("question")
            elif "feature" in answer:
                y_pred.append("feature")
            elif "bug" in answer:
                y_pred.append("bug")
            else: 
                print("없는 레이블?")
                y_pred.append("empty")
                # print(result)
        except Exception as e:
            # print(len(tokenizer(X_test.iloc[i]["text"])['input_ids']))
            y_pred.append("empty")
            # print(e)
            print('bug')
  
    return y_pred

In [12]:
from sklearn.metrics import (accuracy_score, 
                             recall_score, 
                             precision_score, 
                             f1_score)

from transformers import EarlyStoppingCallback, IntervalStrategy

def compute_metrics(p):    
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)    
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [13]:
# open("../CM1/model/")
# output_dir="../CM1/model/epoch3/"+project_name+"-llama-classifier"
output_dir="../CM1/model/0204/"+project_name+"-llama-classifier"
# tokenizer.save_pretrained(output_dir)
# output_dir="facebook_issue_trained_weigths"


peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj",],
)

training_arguments = TrainingArguments(
    output_dir=output_dir,                    # directory to save and repository id
    # 여기
    # num_train_epochs=10,                       # number of training epochs
    num_train_epochs=3,                       # number of training epochs
    per_device_train_batch_size=1,            # batch size per device during training
    gradient_accumulation_steps=1,            # number of steps before performing a backward/update pass
    gradient_checkpointing=True,              # use gradient checkpointing to save memory
    optim="paged_adamw_32bit",
    save_strategy="epoch",
    logging_steps=10,                         # log every 10 steps
    learning_rate=2e-4,                       # learning rate, based on QLoRA paper
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,                        # max gradient norm based on QLoRA paper
    max_steps=-1,
    warmup_ratio=0.03,                        # warmup ratio based on QLoRA paper
    group_by_length=False,
    lr_scheduler_type="cosine",               # use cosine learning rate scheduler
    report_to="tensorboard", 
    # report metrics to tensorboard
    #evaluation_strategy="steps",              # save checkpoint every epoch
    #load_best_model_at_end = True,
    #eval_steps = 25,
    #metric_for_best_model = 'accuracy',

)


trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    #eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=max_seq_length,
    packing=False,
    dataset_kwargs={
        "add_special_tokens": False,
        "append_concat_token": False,
    },
    #compute_metrics=compute_metrics,
    #callbacks = [EarlyStoppingCallback(early_stopping_patience=3)],
)


C:\Users\HJE\anaconda3\envs\ICPC-llama-V2\lib\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length, dataset_kwargs. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

C:\Users\HJE\anaconda3\envs\ICPC-llama-V2\lib\site-packages\trl\trainer\sft_trainer.py:413: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(


In [14]:
trainer.train()
    

Step,Training Loss


TrainOutput(global_step=225, training_loss=0.4426748106214735, metrics={'train_runtime': 3346.3137, 'train_samples_per_second': 0.269, 'train_steps_per_second': 0.067, 'total_flos': 2.982150146772173e+16, 'train_loss': 0.4426748106214735, 'epoch': 3.0})

In [15]:
# trainer.save_model()
# tokenizer.save_pretrained(output_dir)

In [16]:
y_pred = predict(test_data, model, tokenizer)
result=evaluate(y_true, y_pred)
# print(result)

100%|██████████| 300/300 [00:00<00:00, 29992.16it/s]

bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug
bug


In [17]:
evaluation = pd.DataFrame({'text': X_test["text"], 
                           'y_true':y_true, 
                           'y_pred': y_pred},
                          )
# evaluation.to_csv("../CM1/result/metric/epoch3/"+project_name+"_result.csv", index=False)

In [18]:
print(evaluation)

                                                  text    y_true y_pred
0    <|begin_of_text|><|start_header_id|>system<|en...       bug  empty
1    <|begin_of_text|><|start_header_id|>system<|en...       bug  empty
2    <|begin_of_text|><|start_header_id|>system<|en...       bug  empty
3    <|begin_of_text|><|start_header_id|>system<|en...       bug  empty
4    <|begin_of_text|><|start_header_id|>system<|en...       bug  empty
..                                                 ...       ...    ...
295  <|begin_of_text|><|start_header_id|>system<|en...  question  empty
296  <|begin_of_text|><|start_header_id|>system<|en...  question  empty
297  <|begin_of_text|><|start_header_id|>system<|en...  question  empty
298  <|begin_of_text|><|start_header_id|>system<|en...  question  empty
299  <|begin_of_text|><|start_header_id|>system<|en...  question  empty

[300 rows x 3 columns]
